In [6]:
import pandas as pd
import numpy as np
import datetime as dt
import pyproj 
from pyhdf.SD import SD, SDC
import os.path
import os
import calendar
import xarray as xr
import matplotlib.pylab as plt

from sklearn.neighbors import BallTree, ball_tree
from netCDF4 import Dataset

In [ ]:
'''
Download AMSR data from https://seaice.uni-bremen.de/data/amsr2/asi_daygrid_swath/n6250/
'''

In [8]:
# Helper functions

def julian_day_from_date(year, month, day): 
    "Convert date (year,mo,dy) to julian DOY of that day. \
    \n***Altered from SearchMODISDownload by Mike MacFerrin*** \
    \n\nOutput: julian day"
    
    d = dt.date(year, month, day)
    tt = d.timetuple()
    return tt.tm_yday

def get_file_in_directory(path): 
    "Retrieves file names from a directory \
    \n\nInput: path = directory \
    \n\nOutput: list of subdirectories"

    return [name for name in os.listdir(path)
            if os.path.isfile(os.path.join(path, name))]

def JD2_YYYYMMDD(y,jd):
    '''
    Parse dates from julian day (i.e., day of year) to year + month + day
    '''
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
    # We want months (and days) that have less than 10 to represent with a zero
    # i.e., 0405 and not 45
    if month < 10:
        month = str(0) + str(month)
    else:
        month = str(month)
    if jd < 10:
        jd = str(0) + str(jd)
    else:
        jd = str(jd)
    return str(y) + month + jd

In [9]:
def seaiceMask (edate,MODfilelist,SSTfile,amsr_dir,MODoutdir,box):
    '''
    Alignment of MOD29 1km sea ice product to 4km SST grid to produce saved numpy sea ice mask
    # Modified from MODIS Lookup with assistance from Shane Grigsby
    
    Variables:    
    Icefile = string file extension for MODIS file without directory, must already be in directory of these files 
    SSTfile =  string file extension for Terra or Aqua file, make sure satellite matches Icefile coming in
    outfile = string file extension for output numpy array saved
    box = latbounds=box[0], latbounds=box[1], each is an array of min,max bounds
          
    '''
    
    year = edate.year
    mjd = Snow.julian_day_from_date(edate.year, edate.month, edate.day)

    # Set output file names
    outFile = MODoutdir +'/'+ MODfilelist[0][0:10] + str(year) + '%03d.npy' % (mjd)

    if os.path.isfile(outFile):
        print(outFile, 'already exists')
    else:
        # Projections for SST
        inProj2 = pyproj.Proj(init='EPSG:4326') # WGS 84
        outProj2 = pyproj.Proj(init='EPSG:3995') # Arctic Polar Stereo

        altmask = False
        try:
            # Find MOD29 file if one exists for that date
            Icefile = [f for f in MODfilelist if ((year==int(f[-36:-32])) & (mjd==int(f[-32:-29])))][0] # takes out of list

            # Open MODIS file
            mod29 = SD(Icefile, SDC.READ)

            # Set bounding boxes for 1km grid found in metadata for MOD29 1km, these don't change
            ulx=-2383921.6275
            uly=-1430352.9765
            lrx=-1430352.9765
            lry=-2383921.6275
            xdim=951
            ydim=951
            inProj = pyproj.Proj(init='EPSG:3408') # NSIDC northern hemisphere EASE-grid
            outProj = pyproj.Proj(init='EPSG:3995') # Arctic Polar Stereo

            # Create grid for MOD29 in N. Polar Stereo so will have units in m
            londata = np.linspace(ulx, lrx, xdim)
            latdata = np.linspace(uly,lry, ydim)
            lons, lats = np.meshgrid(londata, latdata)
            lon,lat = pyproj.transform(inProj,outProj,lons,lats) 

            # Extract sea ice data 
            sds_obj = mod29.select(0) # select sds
            seaice_data = sds_obj.get() # get sds data

            cloud_data = np.where((seaice_data==50),1,0) # This line has to occur before the seaice_data where line
            # seaice_data overwrites itself-- only 1's and 0's after the next line
            # cloud_data needs to get copied first, otherwise the ==50 line will fail :-/
            seaice_data = np.where((seaice_data==200),1,0) # masking sea ice (200) and cloud (50) just in case, these are given 1s    
            lon_ice = lon[(lon*seaice_data)!=0] # removes lons/lats where sea ice not present, this ravels, no need to do later
            lat_ice = lat[(lat*seaice_data)!=0]
            lon_cloud = lon[(lon*cloud_data)!=0] # Same as above
            lat_cloud = lat[(lat*cloud_data)!=0]

            if len(lat_ice)==0:
                print(('No sea ice pixels in MOD29 for '+str(year)+'%03d') % (mjd))
                altmask = True 
            else:
                # Ravel so BallTree can lookup easily
                # MOD29 sea ice locations
                MOD29_coords = np.vstack([lat_ice,lon_ice]).T  # note y / x switch (i.e., lat long convention)
                # Cloud locations
                cloud_coords = np.vstack([lat_cloud,lon_cloud]).T
        except: 
            print(('No MOD29 for '+str(year)+'%03d') % (mjd))
            altmask = True # this evaluates to 'True' if seaice_data is 'None', i.e., when there is no MOD29


        # Open AMSR file
        # Switches check for appropriate naming convention based on sensor
        # Also check for gap between sensors
        ulx_=0
        uly_=-2000000
        lrx_=500000
        lry_=-3500000

        # Try to find a matching AMSR sea ice file...has missing dates in some of these so will go to except
        try:
            if year > 2012:
                ds = xr.open_dataset(amsr_dir+'asi-AMSR2-n6250-' + JD2_YYYYMMDD(year, mjd) + '-v5.4.nc')
            elif (year == 2012) and (mjd > 183):
                ds = xr.open_dataset(amsr_dir+'asi-AMSR2-n6250-' + JD2_YYYYMMDD(year, mjd) + '-v5.4.nc')
            elif (year == 2011) and (mjd < 278):
                ds = xr.open_dataset(amsr_dir+'asi-n6250-' + JD2_YYYYMMDD(year, mjd) + '-v5.nc')
            elif (year > 2002) and (year < 2011):
                ds = xr.open_dataset(amsr_dir+'asi-n6250-' + JD2_YYYYMMDD(year, mjd) + '-v5.nc')
            elif (year == 2002) and (mjd >= 151):
                ds = xr.open_dataset(amsr_dir+'asi-n6250-' + JD2_YYYYMMDD(year, mjd) + '-v5.nc')
            else:
                amsr = False
        except:
            amsr = False

        # Open AMSR file if it's available
        try:
            df = ds.to_dataframe()
            ds.close()
            tempsub = df.loc[(slice(ulx_,lrx_),slice(lry_,uly_)),'z'].dropna() # Spatial subset (rough)
            ams = tempsub[tempsub >= 5].reset_index() # For sea ice concentrations >5%
            amx_x,ams_y = pyproj.transform(pyproj.Proj(init='EPSG:3411'),outProj2,ams.x.values,ams.y.values)
            amsr_coords = np.vstack([ams_y,amx_x]).T 
            amsr = True
        except: 
            print(('No AMSR mask for '+str(year)+'%03d') % (mjd))


        # Extract coords for SST in desired box
        latbounds = box[0] 
        lonbounds = box[1] 
        fhD = Dataset(SSTfile, mode='r')  
        lonsD = fhD.variables['lon'][:]
        latsD = fhD.variables['lat'][:]
        lat_inds = np.where((latsD > latbounds[0]) & (latsD < latbounds[1]))
        lon_inds = np.where((lonsD > lonbounds[0]) & (lonsD < lonbounds[1]))
        lonSST = lonsD[lon_inds[0]]
        latSST = latsD[lat_inds[0]]

        # Create SST grid in N Polar Stereo
        x1, y1 = np.meshgrid(lonSST,latSST)
        lonSST,latSST = pyproj.transform(inProj2,outProj2,x1,y1)
        SSTcoords = np.vstack([lonSST.ravel(),latSST.ravel()]).T

        # Ravel so BallTree can lookup easily
        # SST locations
        grid_coords = np.vstack([SSTcoords[:,1],SSTcoords[:,0]]).T   # note y / x switch (i.e., lat long convention)       

        # Build lookup, chebyshev = calc dist between lat,lon pairs so can do nearest neighbor in square from SST centerpoints
        # If center coords from MOD29 fall within 2km of SST center, will be counted in query 
        # If just outside radius, partial 1km pixels will not be counted (pixels at edges might have up to 1 pix more sea ice cover than counted due to this)
        # Only does full integers so don't get exact percentage
        if (altmask==True) & (amsr==True):
            # We have AMSR but no MOD29
            square_ball = BallTree(amsr_coords,metric='chebyshev')
            countMOD = square_ball.query_radius(grid_coords, r=3125, count_only=True)
            # Query AMSR coords with SST coords, gives count with nearest neighbor of how many pixels...
            # ...are in a 3.125km box around a SST center point have a AMSR center point
        elif (altmask==True) & (amsr==False):
            print('No MOD29. Also no AMSR. No data for ' + str(year) + '%03d ¯\_(ツ)_/¯') % (mjd)
            return None
        else:
            # We have MOD29
            square_ball = BallTree(MOD29_coords,metric='chebyshev') #sklearn library
            # Query MOD29 coords with SST coords, gives count with nearest neighbor of how many pixels in 2km box around a
            # SST center point have a MOD29 center point
            countMOD = square_ball.query_radius(grid_coords, r=2000, count_only=True) # how do I get counts less than 16 when running with all lat/lons        

        try:
            # If have AMSR
            amsr_ball = BallTree(amsr_coords,metric='chebyshev')
        except:
            None


        # SST center point have a MOD29 center point
        MOD29_mask = np.where(countMOD==0,1,0) # All SST pixels with 1 or more sea ice pix within, turned to 0 as a mask

        try:
            countAMS = amsr_ball.query_radius(cloud_coords, r=3125, count_only=True)
            grid_ball = BallTree(cloud_coords[countAMS > 0],metric='chebyshev')
            cloud_ice = grid_ball.query_radius(grid_coords, r=2000, count_only=True)
            MOD29_mask[cloud_ice>0] = 0
        except:
            None

        MOD29_mask = MOD29_mask.reshape(x1.shape)

        # Save as numpy array
        np.save(outFile, MOD29_mask)

In [13]:
# Create sea ice mask numpy files for the time series

# # For SST v2014
# sat = 'Terra'; st = 'T'
# sat = 'Aqua'; st = 'A'

# For SST v2019
# sat = 'Terra';st = 'TERRA'
sat = 'Aqua'; st = 'AQUA'

MODsearchdir = '/Volumes/LaCieGrande/MODIS_SeaIce_Helheim/GlobalSeaIce'+sat
MODoutdir = '/Volumes/LaCieGrande/MODIS_SeaIce_Helheim/GlobalSeaIce'+sat+'/Aligned'
SSTfile = '/Volumes/Colossus/SST_Helheim_images/HelheimMODISv2019/GlobalSST'+sat+'/'+st+'_MODIS.20030501.L3m.DAY.SST.sst.4km.nc'
AMSRdir = '/Volumes/LaCieGrande/AMSR/'

# if sat=='Terra':
#     time0 = '2000-02-24'
# elif sat=='Aqua':
#     time0 = '2002-07-05'
time0 = '2019-01-01'
end = '2019-12-31'

latbounds = [ 62.4 , 66.7 ] # Serm Shelf only, crop to desired area to make faster
lonbounds = [ -41 , -32.5 ]
box=[latbounds,lonbounds]

MODfilelist = Snow.get_file_in_directory(MODsearchdir)
# Get rid of stupid DS_Store file name if needed
if MODfilelist[0]=='.DS_Store':
    MODfilelist = MODfilelist[1:]
    
os.chdir(MODsearchdir)

# Create icemask for all dates since MODIS first SST acquisition
time_range = pd.date_range(time0, end, freq="D")
for edate in time_range:   
    try:
        seaiceMask(edate,MODfilelist,SSTfile,AMSRdir,MODoutdir,box)
    except Exception as e: 
        print(edate,'failed',e)
        

No AMSR mask for 2019109
No AMSR mask for 2019110
No AMSR mask for 2019111
No sea ice pixels in MOD29 for 2019321
